In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

model = YOLO("./best.pt")

def detect_image(image):
    model = YOLO("./best.pt")  # 모델 로드
    results = model.predict(image, imgsz=1280, save=False)  # YOLO 예측 실행
    
    result_image = results[0].plot()  # 예측 결과 이미지 변환
    return result_image  # 결과 반환

# Gradio UI 설정
demo = gr.Interface(
    fn=detect_image,  # 이미지 분석 함수
    inputs=gr.Image(type="numpy"),  # 입력: 이미지 업로드
    outputs=gr.Image(type="numpy"),  # 출력: 분석된 이미지
    title="YOLO Image Detection",  # UI 제목
    description="Upload an image to detect objects using YOLO model."
)

demo.launch(share=True)  # UI 실행

Outputting Data as a File!

In [13]:
import gradio as gr
import cv2
import numpy as np
import os
import uuid
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("./best.pt")

# 비디오 프레임 처리를 위한 함수
def process_frame(frame):
    """ YOLO 모델을 사용하여 프레임을 감지하고, 바운딩 박스를 그린 후 반환 """
    results = model.predict(frame, imgsz=1280, save=False, device="mps")  # Mac MPS 사용
    result_frame = results[0].plot()

    # 결과 프레임이 float 형식이라면 uint8로 변환
    if result_frame.dtype != np.uint8:
        result_frame = (result_frame * 255).astype(np.uint8)

    return result_frame

# 비디오 감지 함수 (Gradio 스트리밍 지원, FFmpeg 없이)
def detect_video(video_path):
    cap = cv2.VideoCapture(video_path)

    # 비디오 속성 가져오기
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fps = max(1, fps)  # FPS가 0이면 최소 1로 설정

    # UUID 기반으로 고유한 파일명 생성
    output_video_name = f"output_{uuid.uuid4()}.avi"  # AVI로 저장하여 FFmpeg 필요 없음

    # MP4 대신 AVI 사용 (Gradio가 AVI 지원)
    video_codec = cv2.VideoWriter_fourcc(*"XVID")  
    output_video = cv2.VideoWriter(output_video_name, video_codec, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = process_frame(frame)  # YOLO 감지 수행
        output_video.write(processed_frame)  # 결과 저장

    cap.release()
    output_video.release()

    return output_video_name  # 최종 비디오 반환

# Gradio UI 설정
with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center'>YOLO Video Detection</h1>")
    
    with gr.Row():
        with gr.Column():
            video_input = gr.Video(label="Upload Video")
        with gr.Column():
            output_video = gr.File(label="Download Processed Video")  # 파일 다운로드 가능하도록 변경
    
    # 파일 업로드 후 실행
    video_input.upload(
        fn=detect_video,
        inputs=[video_input],
        outputs=[output_video],
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7864


KeyboardInterrupt: 

Real-time detection!

In [23]:
import gradio as gr
import cv2
import numpy as np
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO("./best.pt")

# 비디오 감지 함수 (프레임 스킵 추가)
def detect_video_stream(video_path, frame_skip=2):
    cap = cv2.VideoCapture(video_path)

    frame_count = 0  # 프레임 카운트 초기화
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 스킵 적용 (프레임을 건너뛰면서 처리)
        if frame_count % frame_skip == 0:
            # YOLO 감지 수행
            results = model.predict(frame, imgsz=1280, save=False, device="mps")  # Mac MPS 사용
            result_frame = results[0].plot()

            # OpenCV의 BGR을 RGB로 변환 (Gradio에서 정상 표시되도록)
            result_frame = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)

            # numpy 배열을 반환 (Gradio `gr.Image`에서 정상 표시 가능)
            yield result_frame
        
        frame_count += 1  # 프레임 카운트 증가

    cap.release()

# Gradio UI 설정 (Start AI Detection! 버튼 추가)
with gr.Blocks() as demo:
    gr.HTML("<h1 style='text-align: center; font-size: 24px;'>🚀 YOLO Video Detection (Real-Time Streaming)</h1>")
    
    with gr.Row():
        with gr.Column():
                gr.Markdown("### 🎥 Upload Your Video")
                video_input = gr.Video(label="Upload Video")
                frame_skip_slider = gr.Slider(
                    minimum=1, maximum=10, step=1, value=2, 
                    label="🎞️ Frame Skip (Higher = Faster Processing)"
                )
                start_button = gr.Button("🚀 Start AI Detection!", variant="primary")  # 실행 버튼 추가

        with gr.Column():
                gr.Markdown("### 📊 Processed Video Stream")
                video_output = gr.Image(label="Processed Video", streaming=True, height=480)  # 원래대로 유지
    
    # "Start AI Detection!" 버튼을 클릭하면 감지 실행
    start_button.click(
        fn=detect_video_stream,
        inputs=[video_input, frame_skip_slider],  # 프레임 스킵 옵션 추가
        outputs=[video_output],
    )

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7870

Could not create share link. Missing file: /Users/k23070952/.local/lib/python3.12/site-packages/gradio/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/k23070952/.local/lib/python3.12/site-packages/gradio
